# Caffe学习系列：用训练好的caffemodel来进行分类

Caffe程序自带有一张小猫图片，存放路径为caffe根目录下的examples/images/cat.jpg，如果我们想用一个训练好的caffemodel来对这张图片进行分类，那该怎么办呢？如果不用这张小猫图片，换一张别的图片，又该怎么办呢？如果学会了小猫图片的分类，那么换成其他图片，程序实际上是一样的。

开发caffe的贾大牛团队，利用imagenet图片和caffenet模型训练好了一个caffemodel，供大家下载。要进行图片的分类，这个caffemodel是最好不过的了。所以，不管是用C++来进行分类，还是用python接口来分类，我们都应该准备这样三个文件：

1、caffemodel文件。

可以直接在浏览器里输入地址下载，也可以运行脚本文件下载。点击[下载地址](http://dl.caffe.berkeleyvision.org/bvlc_reference_caffenet.caffemodel)进行下载，文件名称为bvlc_reference_caffenet.caffemodel，文件大小为230M左右，为了代码的统一，将这个caffemodel文件下载到caffe根目录下的models/bvlc_reference_caffenet/文件夹下面。也可以运行脚本文件进行下载：
``` shell
sudo ./scripts/download_model_binary.py models/bvlc_reference_caffenet
```

2、均值文件。

有了caffemodel文件，就需要对应的均值文件，在测试阶段，需要把测试数据减去均值。这个文件我们用脚本来下载，在caffe根目录下执行：
``` shell
sudo sh ./data/ilsvrc12/get_ilsvrc_aux.sh
```
执行并下载后，均值文件放在data/ilsvrc12/文件夹里。

3、synset_words.txt文件

在调用脚本文件下载均值的时候，这个文件也一并下载好了。里面存放的是1000个类的名称。

数据准备好了，我们就可以开始分类了，我们给大家提供两个版本的分类方法：

## 一、C++ 方法

在caffe根目录下的examples/cpp-classification/文件夹下面，有个classification.cpp文件，就是用来分类的。当然编译后，放在/build/examples/cpp_classification/下面。我们就直接运行命令：
``` shell
sudo ./build/examples/cpp_classification/classification.bin \
models/bvlc_reference_caffenet/deploy.prototxt \
models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel \
data/ilsvrc12/imagenet_mean.binaryproto \
data/ilsvrc12/synset_words.txt \
examples/images/cat.jpg
```
命令很长，用了很多的\符号来换行。可以看出，从第二行开始就是参数，每行一个，共需要4个参数。运行成功后，输出top-5结果：
```
---------- Prediction for examples/images/cat.jpg ----------
0.3134 - "n02123045 tabby, tabby cat"
0.2380 - "n02123159 tiger cat"
0.1235 - "n02124075 Egyptian cat"
0.1003 - "n02119022 red fox, Vulpes vulpes"
0.0715 - "n02127052 lynx, catamount"
```
即有0.3134的概率为tabby cat，有0.2380的概率为tiger cat ……

## 二、Python方法

Python接口可以使用jupyter notebook来进行可视化操作，因此推荐使用这种方法。

在这里我就不用可视化了，编写一个py文件，命名为py-classify.py：

In [2]:
# coding=utf-8
# 加载必要的库
import numpy as np
import caffe, sys, os
# 设置当前目录
caffe_root = '/root/Repository/caffe/'
sys.path.insert(0, caffe_root + 'python')
os.chdir(caffe_root)

net_file = caffe_root + 'models/bvlc_reference_caffenet/deploy.prototxt'
caffe_model = caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'
mean_file = caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy'

net = caffe.Net(net_file, caffe_model, caffe.TEST)
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2, 0, 1))
transformer.set_mean('data', np.load(mean_file).mean(1).mean(1))
transformer.set_raw_scale('data', 255)
transformer.set_channel_swap('data', (2, 1, 0))

im = caffe.io.load_image(caffe_root + 'examples/images/cat.jpg')
net.blobs['data'].data[...] = transformer.preprocess('data', im)
out = net.forward()

imagenet_labels_filename = caffe_root + 'data/ilsvrc12/synset_words.txt'
labels = np.loadtxt(imagenet_labels_filename, str, delimiter='\t')

top_k = net.blobs['prob'].data[0].flatten().argsort()[-1:-6:-1]
for i in np.arange(top_k.size):
    print(top_k[i], labels[top_k[i]])

(281, 'n02123045 tabby, tabby cat')
(282, 'n02123159 tiger cat')
(285, 'n02124075 Egyptian cat')
(277, 'n02119022 red fox, Vulpes vulpes')
(287, 'n02127052 lynx, catamount')


caffe开发团队实际上也编写了一个Python版本的分类文件，路径为python/classify.py。

运行这个文件必需两个参数，一个输入图片文件，一个输出结果文件。而且运行必须在Python目录下。假设当前目录是Caffe根目录，则运行：

``` shell
cd python
sudo python classify.py ../examples/images/cat.jpg result.npy
```

分类的结果或保存在当前目录下的result.npy文件里面，是看不见的。有人反映这个文件有错误，运行的时候，会提示：
```
Mean shape incompatible with input shape
```
的错误。因此，要使用这个文件，我们还得进行修改：

1、修改均值计算

定位到：
``` python
mean = np.load(args.mean_file)
```
在这一行下面加上一行：
``` python
mean = mean.mean(1).mean(1)
```
则可以解决报错问题。

2、修改文件，使得结果显示在命令行下：

定位到：
``` python
# classify
start = time.time()
predictions = classfier.predict(inputs, not args.center_only)
print("Done in %.2f s." % (time.time() - start))
```
在这个地方后面加上几行，如下所示：
``` python
imagenet_labels_filename = '../data/ilsvrc12/synset_words.txt'
labels = np.loadtxt(imagenet_labels_filename, str, delimiter='\t')
top_k = predictions.flatten().argsort()[-1:-6:-1]
for i in np.arange(top_k.size):
    print(top_k[i], labels[top_k[i]])
```
这样就可以了，运行后结果会显示在命令行下面，如下：

```
CPU mode
WARNING: Logging before InitGoogleLogging() is written to STDERR
W0127 15:42:50.256335  9004 _caffe.cpp:139] DEPRECATION WARNING - deprecated use of Python interface
W0127 15:42:50.256364  9004 _caffe.cpp:140] Use this instead (with the named "weights" parameter):
W0127 15:42:50.256371  9004 _caffe.cpp:142] Net('../models/bvlc_reference_caffenet/deploy.prototxt', 1, weights='../models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel')
I0127 15:42:50.257944  9004 net.cpp:51] Initializing net from parameters: 
name: "CaffeNet"
state {
  phase: TEST
  level: 0
}
layer {
  name: "data"
  type: "Input"
  top: "data"
  input_param {
    shape {
      dim: 10
      dim: 3
      dim: 227
      dim: 227
    }
  }
}
layer {
  name: "conv1"
  type: "Convolution"
  bottom: "data"
  top: "conv1"
  convolution_param {
    num_output: 96
    kernel_size: 11
    stride: 4
  }
}
layer {
  name: "relu1"
  type: "ReLU"
  bottom: "conv1"
  top: "conv1"
}
layer {
  name: "pool1"
  type: "Pooling"
  bottom: "conv1"
  top: "pool1"
  pooling_param {
    pool: MAX
    kernel_size: 3
    stride: 2
  }
}
layer {
  name: "norm1"
  type: "LRN"
  bottom: "pool1"
  top: "norm1"
  lrn_param {
    local_size: 5
    alpha: 0.0001
    beta: 0.75
  }
}
layer {
  name: "conv2"
  type: "Convolution"
  bottom: "norm1"
  top: "conv2"
  convolution_param {
    num_output: 256
    pad: 2
    kernel_size: 5
    group: 2
  }
}
layer {
  name: "relu2"
  type: "ReLU"
  bottom: "conv2"
  top: "conv2"
}
layer {
  name: "pool2"
  type: "Pooling"
  bottom: "conv2"
  top: "pool2"
  pooling_param {
    pool: MAX
    kernel_size: 3
    stride: 2
  }
}
layer {
  name: "norm2"
  type: "LRN"
  bottom: "pool2"
  top: "norm2"
  lrn_param {
    local_size: 5
    alpha: 0.0001
    beta: 0.75
  }
}
layer {
  name: "conv3"
  type: "Convolution"
  bottom: "norm2"
  top: "conv3"
  convolution_param {
    num_output: 384
    pad: 1
    kernel_size: 3
  }
}
layer {
  name: "relu3"
  type: "ReLU"
  bottom: "conv3"
  top: "conv3"
}
layer {
  name: "conv4"
  type: "Convolution"
  bottom: "conv3"
  top: "conv4"
  convolution_param {
    num_output: 384
    pad: 1
    kernel_size: 3
    group: 2
  }
}
layer {
  name: "relu4"
  type: "ReLU"
  bottom: "conv4"
  top: "conv4"
}
layer {
  name: "conv5"
  type: "Convolution"
  bottom: "conv4"
  top: "conv5"
  convolution_param {
    num_output: 256
    pad: 1
    kernel_size: 3
    group: 2
  }
}
layer {
  name: "relu5"
  type: "ReLU"
  bottom: "conv5"
  top: "conv5"
}
layer {
  name: "pool5"
  type: "Pooling"
  bottom: "conv5"
  top: "pool5"
  pooling_param {
    pool: MAX
    kernel_size: 3
    stride: 2
  }
}
layer {
  name: "fc6"
  type: "InnerProduct"
  bottom: "pool5"
  top: "fc6"
  inner_product_param {
    num_output: 4096
  }
}
layer {
  name: "relu6"
  type: "ReLU"
  bottom: "fc6"
  top: "fc6"
}
layer {
  name: "drop6"
  type: "Dropout"
  bottom: "fc6"
  top: "fc6"
  dropout_param {
    dropout_ratio: 0.5
  }
}
layer {
  name: "fc7"
  type: "InnerProduct"
  bottom: "fc6"
  top: "fc7"
  inner_product_param {
    num_output: 4096
  }
}
layer {
  name: "relu7"
  type: "ReLU"
  bottom: "fc7"
  top: "fc7"
}
layer {
  name: "drop7"
  type: "Dropout"
  bottom: "fc7"
  top: "fc7"
  dropout_param {
    dropout_ratio: 0.5
  }
}
layer {
  name: "fc8"
  type: "InnerProduct"
  bottom: "fc7"
  top: "fc8"
  inner_product_param {
    num_output: 1000
  }
}
layer {
  name: "prob"
  type: "Softmax"
  bottom: "fc8"
  top: "prob"
}
I0127 15:42:50.258764  9004 layer_factory.hpp:77] Creating layer data
I0127 15:42:50.258781  9004 net.cpp:84] Creating Layer data
I0127 15:42:50.258791  9004 net.cpp:380] data -> data
I0127 15:42:50.258812  9004 net.cpp:122] Setting up data
I0127 15:42:50.258824  9004 net.cpp:129] Top shape: 10 3 227 227 (1545870)
I0127 15:42:50.258831  9004 net.cpp:137] Memory required for data: 6183480
I0127 15:42:50.258837  9004 layer_factory.hpp:77] Creating layer conv1
I0127 15:42:50.258850  9004 net.cpp:84] Creating Layer conv1
I0127 15:42:50.258857  9004 net.cpp:406] conv1 <- data
I0127 15:42:50.258864  9004 net.cpp:380] conv1 -> conv1
I0127 15:42:50.258954  9004 net.cpp:122] Setting up conv1
I0127 15:42:50.258965  9004 net.cpp:129] Top shape: 10 96 55 55 (2904000)
I0127 15:42:50.258970  9004 net.cpp:137] Memory required for data: 17799480
I0127 15:42:50.258982  9004 layer_factory.hpp:77] Creating layer relu1
I0127 15:42:50.258992  9004 net.cpp:84] Creating Layer relu1
I0127 15:42:50.258998  9004 net.cpp:406] relu1 <- conv1
I0127 15:42:50.259006  9004 net.cpp:367] relu1 -> conv1 (in-place)
I0127 15:42:50.259016  9004 net.cpp:122] Setting up relu1
I0127 15:42:50.259024  9004 net.cpp:129] Top shape: 10 96 55 55 (2904000)
I0127 15:42:50.259029  9004 net.cpp:137] Memory required for data: 29415480
I0127 15:42:50.259035  9004 layer_factory.hpp:77] Creating layer pool1
I0127 15:42:50.259043  9004 net.cpp:84] Creating Layer pool1
I0127 15:42:50.259049  9004 net.cpp:406] pool1 <- conv1
I0127 15:42:50.259057  9004 net.cpp:380] pool1 -> pool1
I0127 15:42:50.259068  9004 net.cpp:122] Setting up pool1
I0127 15:42:50.259075  9004 net.cpp:129] Top shape: 10 96 27 27 (699840)
I0127 15:42:50.259081  9004 net.cpp:137] Memory required for data: 32214840
I0127 15:42:50.259088  9004 layer_factory.hpp:77] Creating layer norm1
I0127 15:42:50.259095  9004 net.cpp:84] Creating Layer norm1
I0127 15:42:50.259101  9004 net.cpp:406] norm1 <- pool1
I0127 15:42:50.259109  9004 net.cpp:380] norm1 -> norm1
I0127 15:42:50.259119  9004 net.cpp:122] Setting up norm1
I0127 15:42:50.259126  9004 net.cpp:129] Top shape: 10 96 27 27 (699840)
I0127 15:42:50.259132  9004 net.cpp:137] Memory required for data: 35014200
I0127 15:42:50.259138  9004 layer_factory.hpp:77] Creating layer conv2
I0127 15:42:50.259147  9004 net.cpp:84] Creating Layer conv2
I0127 15:42:50.259155  9004 net.cpp:406] conv2 <- norm1
I0127 15:42:50.259160  9004 net.cpp:380] conv2 -> conv2
I0127 15:42:50.259635  9004 net.cpp:122] Setting up conv2
I0127 15:42:50.259647  9004 net.cpp:129] Top shape: 10 256 27 27 (1866240)
I0127 15:42:50.259654  9004 net.cpp:137] Memory required for data: 42479160
I0127 15:42:50.259662  9004 layer_factory.hpp:77] Creating layer relu2
I0127 15:42:50.259670  9004 net.cpp:84] Creating Layer relu2
I0127 15:42:50.259675  9004 net.cpp:406] relu2 <- conv2
I0127 15:42:50.259683  9004 net.cpp:367] relu2 -> conv2 (in-place)
I0127 15:42:50.259692  9004 net.cpp:122] Setting up relu2
I0127 15:42:50.259701  9004 net.cpp:129] Top shape: 10 256 27 27 (1866240)
I0127 15:42:50.259706  9004 net.cpp:137] Memory required for data: 49944120
I0127 15:42:50.259711  9004 layer_factory.hpp:77] Creating layer pool2
I0127 15:42:50.259717  9004 net.cpp:84] Creating Layer pool2
I0127 15:42:50.259723  9004 net.cpp:406] pool2 <- conv2
I0127 15:42:50.259730  9004 net.cpp:380] pool2 -> pool2
I0127 15:42:50.259739  9004 net.cpp:122] Setting up pool2
I0127 15:42:50.259747  9004 net.cpp:129] Top shape: 10 256 13 13 (432640)
I0127 15:42:50.259753  9004 net.cpp:137] Memory required for data: 51674680
I0127 15:42:50.259759  9004 layer_factory.hpp:77] Creating layer norm2
I0127 15:42:50.259768  9004 net.cpp:84] Creating Layer norm2
I0127 15:42:50.259773  9004 net.cpp:406] norm2 <- pool2
I0127 15:42:50.259781  9004 net.cpp:380] norm2 -> norm2
I0127 15:42:50.259789  9004 net.cpp:122] Setting up norm2
I0127 15:42:50.259796  9004 net.cpp:129] Top shape: 10 256 13 13 (432640)
I0127 15:42:50.259801  9004 net.cpp:137] Memory required for data: 53405240
I0127 15:42:50.259807  9004 layer_factory.hpp:77] Creating layer conv3
I0127 15:42:50.259815  9004 net.cpp:84] Creating Layer conv3
I0127 15:42:50.259821  9004 net.cpp:406] conv3 <- norm2
I0127 15:42:50.259830  9004 net.cpp:380] conv3 -> conv3
I0127 15:42:50.261205  9004 net.cpp:122] Setting up conv3
I0127 15:42:50.261222  9004 net.cpp:129] Top shape: 10 384 13 13 (648960)
I0127 15:42:50.261227  9004 net.cpp:137] Memory required for data: 56001080
I0127 15:42:50.261240  9004 layer_factory.hpp:77] Creating layer relu3
I0127 15:42:50.261247  9004 net.cpp:84] Creating Layer relu3
I0127 15:42:50.261253  9004 net.cpp:406] relu3 <- conv3
I0127 15:42:50.261265  9004 net.cpp:367] relu3 -> conv3 (in-place)
I0127 15:42:50.261273  9004 net.cpp:122] Setting up relu3
I0127 15:42:50.261279  9004 net.cpp:129] Top shape: 10 384 13 13 (648960)
I0127 15:42:50.261284  9004 net.cpp:137] Memory required for data: 58596920
I0127 15:42:50.261291  9004 layer_factory.hpp:77] Creating layer conv4
I0127 15:42:50.261298  9004 net.cpp:84] Creating Layer conv4
I0127 15:42:50.261304  9004 net.cpp:406] conv4 <- conv3
I0127 15:42:50.261313  9004 net.cpp:380] conv4 -> conv4
I0127 15:42:50.262269  9004 net.cpp:122] Setting up conv4
I0127 15:42:50.262281  9004 net.cpp:129] Top shape: 10 384 13 13 (648960)
I0127 15:42:50.262286  9004 net.cpp:137] Memory required for data: 61192760
I0127 15:42:50.262295  9004 layer_factory.hpp:77] Creating layer relu4
I0127 15:42:50.262305  9004 net.cpp:84] Creating Layer relu4
I0127 15:42:50.262312  9004 net.cpp:406] relu4 <- conv4
I0127 15:42:50.262320  9004 net.cpp:367] relu4 -> conv4 (in-place)
I0127 15:42:50.262327  9004 net.cpp:122] Setting up relu4
I0127 15:42:50.262334  9004 net.cpp:129] Top shape: 10 384 13 13 (648960)
I0127 15:42:50.262341  9004 net.cpp:137] Memory required for data: 63788600
I0127 15:42:50.262346  9004 layer_factory.hpp:77] Creating layer conv5
I0127 15:42:50.262354  9004 net.cpp:84] Creating Layer conv5
I0127 15:42:50.262362  9004 net.cpp:406] conv5 <- conv4
I0127 15:42:50.262368  9004 net.cpp:380] conv5 -> conv5
I0127 15:42:50.262995  9004 net.cpp:122] Setting up conv5
I0127 15:42:50.263006  9004 net.cpp:129] Top shape: 10 256 13 13 (432640)
I0127 15:42:50.263012  9004 net.cpp:137] Memory required for data: 65519160
I0127 15:42:50.263022  9004 layer_factory.hpp:77] Creating layer relu5
I0127 15:42:50.263029  9004 net.cpp:84] Creating Layer relu5
I0127 15:42:50.263036  9004 net.cpp:406] relu5 <- conv5
I0127 15:42:50.263046  9004 net.cpp:367] relu5 -> conv5 (in-place)
I0127 15:42:50.263053  9004 net.cpp:122] Setting up relu5
I0127 15:42:50.263061  9004 net.cpp:129] Top shape: 10 256 13 13 (432640)
I0127 15:42:50.263067  9004 net.cpp:137] Memory required for data: 67249720
I0127 15:42:50.263072  9004 layer_factory.hpp:77] Creating layer pool5
I0127 15:42:50.263079  9004 net.cpp:84] Creating Layer pool5
I0127 15:42:50.263085  9004 net.cpp:406] pool5 <- conv5
I0127 15:42:50.263092  9004 net.cpp:380] pool5 -> pool5
I0127 15:42:50.263105  9004 net.cpp:122] Setting up pool5
I0127 15:42:50.263113  9004 net.cpp:129] Top shape: 10 256 6 6 (92160)
I0127 15:42:50.263118  9004 net.cpp:137] Memory required for data: 67618360
I0127 15:42:50.263123  9004 layer_factory.hpp:77] Creating layer fc6
I0127 15:42:50.263137  9004 net.cpp:84] Creating Layer fc6
I0127 15:42:50.263144  9004 net.cpp:406] fc6 <- pool5
I0127 15:42:50.263151  9004 net.cpp:380] fc6 -> fc6
I0127 15:42:50.320396  9004 net.cpp:122] Setting up fc6
I0127 15:42:50.320433  9004 net.cpp:129] Top shape: 10 4096 (40960)
I0127 15:42:50.320442  9004 net.cpp:137] Memory required for data: 67782200
I0127 15:42:50.320485  9004 layer_factory.hpp:77] Creating layer relu6
I0127 15:42:50.320500  9004 net.cpp:84] Creating Layer relu6
I0127 15:42:50.320508  9004 net.cpp:406] relu6 <- fc6
I0127 15:42:50.320518  9004 net.cpp:367] relu6 -> fc6 (in-place)
I0127 15:42:50.320529  9004 net.cpp:122] Setting up relu6
I0127 15:42:50.320538  9004 net.cpp:129] Top shape: 10 4096 (40960)
I0127 15:42:50.320545  9004 net.cpp:137] Memory required for data: 67946040
I0127 15:42:50.320552  9004 layer_factory.hpp:77] Creating layer drop6
I0127 15:42:50.320566  9004 net.cpp:84] Creating Layer drop6
I0127 15:42:50.320586  9004 net.cpp:406] drop6 <- fc6
I0127 15:42:50.320595  9004 net.cpp:367] drop6 -> fc6 (in-place)
I0127 15:42:50.320607  9004 net.cpp:122] Setting up drop6
I0127 15:42:50.320614  9004 net.cpp:129] Top shape: 10 4096 (40960)
I0127 15:42:50.320629  9004 net.cpp:137] Memory required for data: 68109880
I0127 15:42:50.320634  9004 layer_factory.hpp:77] Creating layer fc7
I0127 15:42:50.320643  9004 net.cpp:84] Creating Layer fc7
I0127 15:42:50.320649  9004 net.cpp:406] fc7 <- fc6
I0127 15:42:50.320655  9004 net.cpp:380] fc7 -> fc7
I0127 15:42:50.345847  9004 net.cpp:122] Setting up fc7
I0127 15:42:50.345880  9004 net.cpp:129] Top shape: 10 4096 (40960)
I0127 15:42:50.345888  9004 net.cpp:137] Memory required for data: 68273720
I0127 15:42:50.345901  9004 layer_factory.hpp:77] Creating layer relu7
I0127 15:42:50.345912  9004 net.cpp:84] Creating Layer relu7
I0127 15:42:50.345921  9004 net.cpp:406] relu7 <- fc7
I0127 15:42:50.345932  9004 net.cpp:367] relu7 -> fc7 (in-place)
I0127 15:42:50.345943  9004 net.cpp:122] Setting up relu7
I0127 15:42:50.345950  9004 net.cpp:129] Top shape: 10 4096 (40960)
I0127 15:42:50.345955  9004 net.cpp:137] Memory required for data: 68437560
I0127 15:42:50.345960  9004 layer_factory.hpp:77] Creating layer drop7
I0127 15:42:50.345969  9004 net.cpp:84] Creating Layer drop7
I0127 15:42:50.345975  9004 net.cpp:406] drop7 <- fc7
I0127 15:42:50.345981  9004 net.cpp:367] drop7 -> fc7 (in-place)
I0127 15:42:50.345989  9004 net.cpp:122] Setting up drop7
I0127 15:42:50.345995  9004 net.cpp:129] Top shape: 10 4096 (40960)
I0127 15:42:50.345999  9004 net.cpp:137] Memory required for data: 68601400
I0127 15:42:50.346005  9004 layer_factory.hpp:77] Creating layer fc8
I0127 15:42:50.346012  9004 net.cpp:84] Creating Layer fc8
I0127 15:42:50.346017  9004 net.cpp:406] fc8 <- fc7
I0127 15:42:50.346024  9004 net.cpp:380] fc8 -> fc8
I0127 15:42:50.352356  9004 net.cpp:122] Setting up fc8
I0127 15:42:50.352381  9004 net.cpp:129] Top shape: 10 1000 (10000)
I0127 15:42:50.352386  9004 net.cpp:137] Memory required for data: 68641400
I0127 15:42:50.352397  9004 layer_factory.hpp:77] Creating layer prob
I0127 15:42:50.352432  9004 net.cpp:84] Creating Layer prob
I0127 15:42:50.352439  9004 net.cpp:406] prob <- fc8
I0127 15:42:50.352448  9004 net.cpp:380] prob -> prob
I0127 15:42:50.352466  9004 net.cpp:122] Setting up prob
I0127 15:42:50.352473  9004 net.cpp:129] Top shape: 10 1000 (10000)
I0127 15:42:50.352478  9004 net.cpp:137] Memory required for data: 68681400
I0127 15:42:50.352483  9004 net.cpp:200] prob does not need backward computation.
I0127 15:42:50.352490  9004 net.cpp:200] fc8 does not need backward computation.
I0127 15:42:50.352497  9004 net.cpp:200] drop7 does not need backward computation.
I0127 15:42:50.352504  9004 net.cpp:200] relu7 does not need backward computation.
I0127 15:42:50.352509  9004 net.cpp:200] fc7 does not need backward computation.
I0127 15:42:50.352515  9004 net.cpp:200] drop6 does not need backward computation.
I0127 15:42:50.352521  9004 net.cpp:200] relu6 does not need backward computation.
I0127 15:42:50.352527  9004 net.cpp:200] fc6 does not need backward computation.
I0127 15:42:50.352533  9004 net.cpp:200] pool5 does not need backward computation.
I0127 15:42:50.352540  9004 net.cpp:200] relu5 does not need backward computation.
I0127 15:42:50.352545  9004 net.cpp:200] conv5 does not need backward computation.
I0127 15:42:50.352551  9004 net.cpp:200] relu4 does not need backward computation.
I0127 15:42:50.352558  9004 net.cpp:200] conv4 does not need backward computation.
I0127 15:42:50.352565  9004 net.cpp:200] relu3 does not need backward computation.
I0127 15:42:50.352571  9004 net.cpp:200] conv3 does not need backward computation.
I0127 15:42:50.352591  9004 net.cpp:200] norm2 does not need backward computation.
I0127 15:42:50.352599  9004 net.cpp:200] pool2 does not need backward computation.
I0127 15:42:50.352615  9004 net.cpp:200] relu2 does not need backward computation.
I0127 15:42:50.352622  9004 net.cpp:200] conv2 does not need backward computation.
I0127 15:42:50.352636  9004 net.cpp:200] norm1 does not need backward computation.
I0127 15:42:50.352643  9004 net.cpp:200] pool1 does not need backward computation.
I0127 15:42:50.352649  9004 net.cpp:200] relu1 does not need backward computation.
I0127 15:42:50.352654  9004 net.cpp:200] conv1 does not need backward computation.
I0127 15:42:50.352661  9004 net.cpp:200] data does not need backward computation.
I0127 15:42:50.352668  9004 net.cpp:242] This network produces output prob
I0127 15:42:50.352680  9004 net.cpp:255] Network initialization done.
I0127 15:42:50.851881  9004 upgrade_proto.cpp:44] Attempting to upgrade input file specified using deprecated transformation parameters: ../models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel
I0127 15:42:50.851919  9004 upgrade_proto.cpp:47] Successfully upgraded file specified using deprecated data transformation parameters.
W0127 15:42:50.851925  9004 upgrade_proto.cpp:49] Note that future Caffe releases will only support transform_param messages for transformation fields.
I0127 15:42:50.851930  9004 upgrade_proto.cpp:53] Attempting to upgrade input file specified using deprecated V1LayerParameter: ../models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel
I0127 15:42:51.254047  9004 upgrade_proto.cpp:61] Successfully upgraded file specified using deprecated V1LayerParameter
I0127 15:42:51.312163  9004 net.cpp:744] Ignoring source layer loss
Loading file: ../examples/images/cat.jpg
Classifying 1 inputs.
Done in 0.58 s.
(281, 'n02123045 tabby, tabby cat')
(282, 'n02123159 tiger cat')
(285, 'n02124075 Egyptian cat')
(287, 'n02127052 lynx, catamount')
(278, 'n02119789 kit fox, Vulpes macrotis')
Saving results into result.npy
```